Author: Zhanchen Guo

Note that this program is for Python 3

In [61]:
from copy import deepcopy

class Literal:
    
    def __init__(self, name, sign=True):
        self.name = name
        self.sign = sign
    
    def __neg__(self):
        return Literal(self.name, not self.sign)
    
    def __eq__(self, obj):
        return obj.name == self.name 
    
    def __str__(self):
        return "-" + self.name if not self.sign else self.name

    def __repr__(self):
        return self.__str__()
    
    def __hash__(self):
        return hash(self.name + str(self.sign)) 
    
A = Literal("A")
B = Literal("B")
C = Literal("C")
D = Literal("D")
E = Literal("E")
F = Literal("F")
false = Literal("false")

### Define Literal class and initalized the letters and the false objects

Literal only contain name and the sign

`__neg__`
return a Literal that has the same name but opposite sign, so -"A" will return "-A"

`__eq__`
this is for the following case when we try symbol == key 
it only compares the name, ignoring the sign

`__str__`
return the name when str() called. -A will return "-A"

`__repr__`
in the case of print(), it will call the str version of the liberal

`__hash__`
for the case in distionary and set, this way -A will be different than A. Notice that this cause the following function has to check the A, and -A case as key

In [62]:
def _add_to_model(model, symbol):
    if symbol.sign:
        model[symbol] = True
    else:
        model[-symbol] = False
    return model

def find_pure_symbol(KB):
    symbols = set()
    for clause in KB:
        symbols.update(clause)
    
    model = {}
    for symbol in symbols:
        if not (-symbol in symbols):
            _add_to_model(model, symbol)
    return model

def _model_propogate(KB, model):
    for key, value in model.items():
        for i, clause in enumerate(KB):
            for symbol in list(clause):
                if symbol == key:
                    if (value and symbol.sign) or (not value and not symbol.sign): # same sign
                        KB[i] = set([])
                    else:
                        clause.remove(symbol)  
                        if not clause:
                            KB[i] = set([false])
                        else:
                            KB[i] = clause
    return KB
    
def find_unit_clause(KB):
    model = {}
    # find out all unit clauses and add them to the model 
    for clause in KB:
        if len(clause) == 1 and not (false in clause):
            symbol = clause.pop()
            _add_to_model(model, symbol)
    return model

def pickone(KB):
    for clause in KB:
        if clause and not (false in clause):
            picked = deepcopy(list(clause)[0]) # very important. Otherwise it will mess up the sign in the clause
            picked.sign = True #standardized to pick the positive version of literal
            return True, picked
        if false in clause:
            return False, "failed"
    return False, "none"

# KB = [{A,B,D},{-C},{-A,B,D}]
# print(find_pure_symbol(KB))
# print(find_pure_symbol(KB))

## necessary functions

`_add_to_model`

Note that since `__hash__` distinguish the sign, we want to make sure we always add a positive literal into dictionary, so this is a helper function that if symbol A is True, add A:True to model,and if symbol -A should be True, then add A:False. This function is especially for the unit clause case. 

`find_pure_symbol`

The first loop find all the symbol into a set, notice that since hash distingush the sign, A and -A will both be in. That is also the beauty of it because you can check if opposite sign symbol exist in a set. If so, it is not a pure symbol (see the second loop)

`_model_propogate`

propogate the given model into KB, and return the resulting KB. If it is the same sign in clause compare to the value, the entire clause is true, therefore replace it with empty set. If it is the opposite sign then remove it from the list as the symbol will end up as false. If that is also the only symbol in clause, the entire clause replace with false object. Notice that since the argument passed by refernce. even if we do not catch the return, KB will changed. This is also some times we need deepcopy in case we do not mess up with the original KB. 

`find_unit_clause`

find all the clause that length is 1 yet not as false object. 

`pickone`

pick the first symbol in the clause in KB. Make sure the sign is positive

In [63]:
def DPLLSatisfiable(KB):
    count = 0
    for clause in KB:
        if false in clause:
            return False, None
        if not clause:
            count += 1
    if count == len(KB):
        return True, {}
    
    model = {}
    pure_model = find_pure_symbol(KB)
    _model_propogate(KB, pure_model)

    model.update(pure_model)
    
    unit_model = find_unit_clause(KB)
    _model_propogate(KB, unit_model)
    model.update(unit_model)

    success, picked = pickone(KB)
    if not success:
        if picked == "none":
            return True, model
        else:
            return False, None
        
    model1 = deepcopy(model)
    model2 = deepcopy(model)
    model1[picked] = True
    model2[picked] = False
    
    leftvalid, leftmodel = DPLLSatisfiable(_model_propogate(deepcopy(KB), model1))
    if leftvalid:
        model1.update(leftmodel)
        return True, model1

    rightvalid, rightmodel = DPLLSatisfiable(_model_propogate(deepcopy(KB), model2))
    if rightvalid:
        model2.update(rightmodel) 
        return True, model2
    return False, None

def if_free(KB, model):
    symbols = set([])
    for clause in KB:
        for symbol in list(clause):
            picked = deepcopy(symbol)
            picked.sign = True
            if not (picked in list(symbols)):
                symbols.add(picked)
    for symbol in list(symbols):
        if not (symbol in model):
            model[symbol] = "free"
    return model

## main functions

`DPLLSatisfiable`

Implementation for the DPLL algorithm. Firstly check if the KB is all empty (final state). Secondly check both pure symbol and unit clause, applying to KB and add them to the model. Thirdly, picked a symbol, if there is no symbol to pick from, then it is finished. Lastly, set a recursive called for both case of the picked symbol be True and False. Notice that in order to get the final model. I choose to return the model in this function to pass onto the upper level if it return True. 

`if_free`

Model returned by DPLLSatisfiable function ignored the literal that can be either True and False yet not impact the result of the KB. if_free checks what are these symbols and add them as "free" to model and return. 

```
S1:A ⇔ (B∨E). 
    A ⇒ (B ∨ E) ∧ (B ∨ E) ⇒ A
    ¬A ∨ (B ∨ E) ∧ (¬ (B ∨ E) ∨ A)
    ¬A ∨ B ∨ E ∧ ((¬B ∧ ¬E) ∨ A)
    (¬A ∨ B ∨ E) ∧ (¬B ∨ A) ∧ (¬E ∨ A)
S2: E ⇒ D.
    ¬E ∨ D
S3: C ∧ F ⇒ ¬B. 
    ¬(C ∧ F) ∨ ¬B
    ¬C ∨ ¬F ∨ ¬B
S4: E ⇒ B.
    ¬E ∨ B
S5: B ⇒ F. 
    ¬B ∨ F
S6: B ⇒ C
    ¬B ∨ C
```
Therefore: (¬A ∨ B ∨ E) ∧ (¬B ∨ A) ∧ (¬E ∨ A) ∧ (¬E ∨ D) ∧ (¬C ∨ ¬F ∨ ¬B) ∧ (¬E ∨ B) ∧ (¬B ∨ F) ∧ (¬B ∨ C)


Readable code: ```[{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]```

In [64]:
KB = [{A,B,D},{-C},{-A,B,D}]
KB1 = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]
KB2 = deepcopy(KB1)
_, models = DPLLSatisfiable(KB1)
print(models)
print(if_free(KB2, models))

{D: True, B: False, E: False, A: False}
{D: True, B: False, E: False, A: False, F: 'free', C: 'free'}
